In [1]:
import geopandas as gpd
import pandas as pd
import folium
import numpy as np
import panel as pn
pn.extension()
pd.options.mode.chained_assignment = None

In [2]:
import pickle
with open('one_obj_to_rule_them_all.pkl', 'rb') as p:
    obj = pickle.load(p)

data = obj['data']
meta_snowex = obj['meta']['snowex']
meta_awdb = obj['meta']['awdb']
data['difference_swe'] = data['swe_awdb'] - data['swe_snowex']
data['difference_depth'] = data['depth_awdb'] - data['depth_snowex']
data.index = data.index.astype(str)
data['date'] = data['date'].astype(str)
data.head()

Index(['snowex_pit_id', 'instrument', 'snowex_id', 'geometry', 'swe_snowex',
       'depth_snowex', 'date', 'swe_awdb', 'depth_awdb', 'awdb_id', 'distance',
       'difference_swe', 'difference_depth'],
      dtype='object')

In [5]:
cmap_col = 'distance'

# Create map object
map_x = meta_awdb.dissolve().centroid.x[0]
map_y = meta_awdb.dissolve().centroid.y[0]
m = folium.Map(location=[map_y, map_x], zoom_start=5, tiles=None, 
                     control_scale=True)
folium.TileLayer('Stamen terrain', name='Terrain basemap', control=True).add_to(m)
folium.TileLayer('CartoDB positron', name='Light basemap', control=True).add_to(m)


# Colormap for points
from branca.colormap import linear as linear_cm
if 'difference' in cmap_col:
    # Divergent colormap
    cmap = linear_cm.RdBu_09.scale(min(data[cmap_col]), max(data[cmap_col]))
else:
    # Sequential colormap
    cmap = linear_cm.YlGnBu_09.scale(min(data[cmap_col]), max(data[cmap_col]))
    
viz_colors = [cmap(i) if not pd.isnull(i) else 'black' for i in data.loc[data['instrument']=='manual', cmap_col]]


# Clean up columns for tooltip
old_cols = data.columns
new_cols = ['Pit ID','Instrument','SnowEx site','geometry','SWE (pit, mm)','Snow depth (pit, mm)',
            'Date collected','SWE (AWDB station, mm)','Snow depth (AWDB station, mm)',
            'AWDB Station ID','Distance from pit to station (m)', 'SWE difference (mm)',
            'Snow depth difference (mm)']
data.columns = new_cols
tooltip_cols = ['Date collected','SnowEx site','Pit ID','Snow depth (pit, mm)','SWE (pit, mm)',
                'AWDB Station ID','Snow depth (AWDB station, mm)','SWE (AWDB station, mm)',
                'Snow depth difference (mm)','SWE difference (mm)','Distance from pit to station (m)']

# Explore layers
awdb_layer = meta_awdb.explore(
    name='AWDB stations', 
    color='red',
    m=m,
    tooltip=['site_name', 'site_id'],
    marker_kwds={'radius': 10},
    style_kwds={'color': '#4a4a4a','weight': 1}
)

snowex_layer = meta_snowex.explore(
    name='All SnowEx pits', 
    color='orange',
    m=m,
    tooltip=['site_name', 'site_id'],
    marker_kwds={'radius': 10},
    style_kwds={'color': '#4a4a4a','weight': 1}
)

pit_layer = data[data['Instrument']=='manual'].explore(
    name='Pit data',
    color=viz_colors,
    m=m,
    tooltip=tooltip_cols,
    marker_kwds={'radius': 10},
    style_kwds={'color': '#4a4a4a','weight': 1}
)

data.columns = old_cols

# Lidar layer
import rasterio
with rasterio.open('gm_2022_02_02_depth.tif') as src:
    lidar = src.read()
    lidar_bounds = src.bounds
    lidar_crs = src.crs
    
# lidar_layer = folium.raster_layers.ImageOverlay(
#     image=lidar[0],
#     bounds=lidar_bounds,
#     name='Lidar data'
# ).add_to(m)


m.add_child(folium.LayerControl())
cmap.caption = cmap_col
cmap.add_to(m)
m


/tmp/ipykernel_820/1807660329.py:4: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  map_x = meta_awdb.dissolve().centroid.x[0]
/tmp/ipykernel_820/1807660329.py:5: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  map_y = meta_awdb.dissolve().centroid.y[0]


In [7]:
meta_awdb.dissolve().centroid.x[0]

/tmp/ipykernel_820/1041152991.py:1: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  meta_awdb.dissolve().centroid.x[0]


-113.08008249999999

In [5]:
for idx in meta_awdb.index:
    print(idx)

155
756
589
197
483
557
636
470
320
660
550
688
307
732
556
877
564
253
98
71
118
33
50
64
114
473
471
472
